In [ ]:
# using Images, Colors, FixedPointNumbers, ImageView
# using ImageMagick

In [1]:
using PyCall
@pyimport mayavi.mlab as mlab

pygui_start(:qt) # make mayavi non-blocking

mlab.figure(1, bgcolor=(1, 1, 1), fgcolor=(0, 0, 0), size=(1000, 800));

2016-01-16 10:19:39.113 julia[88978:2201046] *** WARNING: Method userSpaceScaleFactor in class NSView is deprecated on 10.7 and later. It should not be used in new applications. Use convertRectToBacking: instead. 


In [2]:
include("ndgrid_rak.jl")

ndgrid (generic function with 3 methods)

In [3]:
num = 50
x = linspace(-10.0,10.0,num)
y = linspace(-10.0,10.0,num)
z = linspace(-10.0,10.0,num)
nothing

In [4]:
X, Y, Z = ndgrid(x, y, z)

# create a 3D phantom
S1 = 300./(X.^2+2.0*Y.^2+0.5*Z.^2+3.0)+
    200./(0.5*(X-2.5).^2+0.5*Y.^2+0.5*Z.^2+1.0);

In [5]:
# S2 = 150./(0.5*(X-5).^2+0.5*(Y-5).^2+0.5*(Z-5).^2+1.0);

In [6]:
# function f(x)
#     if abs(x)<5.0 x=0.0 end
#     return x
# end
# nothing

# map!((x) -> f(x) , S);

In [9]:
isosurf = mlab.contour3d(S1, vmin=0.0, colormap="cool", transparent=true)

PyObject <mayavi.modules.iso_surface.IsoSurface object at 0x3293062f0>

In [11]:
isosurf[:contour][:number_of_contours] = 12
isosurf[:actor][:property][:opacity] = 0.3
isosurf[:contour][:minimum_contour] = 20.0
isosurf[:contour][:maximum_contour] = maximum(S1)
mlab.draw()

In [7]:
for i=4:0.025:2
    mlab.clf()
    S2 = 150./(0.5*(X-1.0*i).^2+0.5*(Y-1.0*i).^2+0.5*(Z-1.0*i).^2+1.0);
    S = S1 + S2
    isosurf = mlab.contour3d(S, vmin=0.0, colormap="cool", transparent=true)

    isosurf[:contour][:number_of_contours] = 12
    isosurf[:actor][:property][:opacity] = 0.3
    isosurf[:contour][:minimum_contour] = 20.0
    isosurf[:contour][:maximum_contour] = maximum(S)
    mlab.draw()
    sleep(0.02)
end
nothing

In [ ]:
# a=Images.load("grace.png")

In [ ]:
# img = testimage("mandrill")

In [1]:
# ;ls -la

In [8]:
S = S1 + S2

LoadError: LoadError: UndefVarError: S2 not defined
while loading In[8], in expression starting on line 1